# pip install xgboost

# 데이터셋

In [1]:
# pip install pandas numpy scikit-learn xgboost

import pandas as pd
import numpy as np

np.random.seed(42)

# 샘플 개수 (예: 500명)
n = 500

# 1. 가상 환자 특성 생성
data = pd.DataFrame({
    'age': np.random.randint(20, 90, n),                 # 나이
    'systolic_bp': np.random.normal(140, 25, n),         # 수축기 혈압
    'diastolic_bp': np.random.normal(85, 15, n),         # 이완기 혈압
    'platelet': np.random.normal(250000, 60000, n),      # 혈소판 수
    'inr': np.random.normal(1.1, 0.3, n),                # INR (혈액 응고 정도)
    'gcs': np.random.randint(8, 15, n),                  # GCS (의식 상태)
    'headache_duration': np.random.uniform(0, 24, n),    # 두통 지속 시간(시간)
    'hypertension': np.random.choice([0,1], n, p=[0.4,0.6]),  # 고혈압 병력
    'trauma_history': np.random.choice([0,1], n, p=[0.7,0.3]) # 외상 여부
})

# 2. "출혈 발생(label)" 생성 (임의의 논리로)
# 혈압↑, INR↑, 혈소판↓, GCS↓ → 출혈 확률↑
risk = (
    0.02*(data['systolic_bp'] - 120)
    + 0.5*(data['inr'] - 1)
    - 0.00001*(data['platelet'] - 200000)
    - 0.1*(data['gcs'] - 15)
    + 0.5*data['hypertension']
    + 0.3*data['trauma_history']
)
prob = 1 / (1 + np.exp(-risk))
data['label'] = np.random.binomial(1, prob)

# 3. 파일 저장
data.to_csv("synthetic_brain_hemorrhage_data.csv", index=False)

print(data.head())
print("\n출혈 환자 비율:", data['label'].mean())


   age  systolic_bp  diastolic_bp       platelet       inr  gcs  \
0   71   176.093680     97.473141  206034.782219  1.339476   13   
1   34    92.127316     88.114190  207899.685399  1.041242   14   
2   80   161.298612     82.762163  279438.173285  0.928956   10   
3   40   159.208486     75.108799  188325.834057  1.285545    9   
4   43   153.867320     77.854334  278236.156047  1.500742   14   

   headache_duration  hypertension  trauma_history  label  
0          20.519376             0               0      1  
1          21.181437             1               0      0  
2          11.191434             1               0      1  
3           1.816793             0               0      1  
4           9.307826             1               1      0  

출혈 환자 비율: 0.666


# xgboost

XGBoost는 “수치형 데이터(tabular data)”를 다룰 때
정확도·속도·안정성·해석력이 모두 우수한 머신러닝 알고리즘입니다.

성능이 뛰어나다	Gradient Boosting(부스팅) 방식을 사용해, 단순 트리·랜덤포레스트보다 예측 정확도가 높습니다. 실제로 Kaggle 같은 대회에서도 수치 데이터 부문 1~2위를 다툼.	

2️⃣ 결측값 자동 처리	데이터에 NaN이 있어도 스스로 최적 방향을 찾아 처리 → 전처리 부담 ↓	

3️⃣ 변수 중요도 해석 가능	각 변수(예: 혈압, INR 등)가 예측에 얼마나 기여했는지 확인 가능 → 의료 분야에서 ‘설명 가능한 AI(XAI)’로 활용 용이	

4️⃣ 불균형 데이터 대응 우수	scale_pos_weight, eval_metric='auc' 등 옵션으로 출혈/비출혈 비율이 달라도 성능 유지	

5️⃣ 빠르고 효율적	C++ 기반으로 구현돼 있고, CPU·GPU 병렬 처리 지원 → 학습 속도가 매우 빠름	

**** 다른 알고리즘과 비교

모델	장점	한계

Logistic Regression	해석이 쉬움	단순한 선형 관계만 표현 가능

Decision Tree	이해 쉽고 빠름	과적합 잘 일어남

Random Forest	안정적, 과적합 줄임	트리 간 독립 → 학습 효율 낮음

XGBoost	🔥 높은 성능 + 빠름 + 해석 가능	하이퍼파라미터 많아 튜닝 필요

딥러닝 (MLP)	대규모 데이터, 복잡한 패턴에 강함	적은 데이터에서는 과적합 위험, 해석 어려움

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from xgboost import XGBClassifier

# 1. 데이터 로드
data = pd.read_csv("synthetic_brain_hemorrhage_data.csv")

# 2. 입력(X), 정답(y) 분리
X = data.drop("label", axis=1)
y = data["label"]

# 3. 학습용 / 테스트용 분리 (비율 유지)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. XGBoost 모델 정의 및 학습
model = XGBClassifier(
    n_estimators=300,      # 트리 개수
    learning_rate=0.05,    # 학습률
    max_depth=4,           # 트리 깊이
    subsample=0.8,         # 데이터 샘플 비율
    colsample_bytree=0.8,  # 특성 샘플 비율

    # 학습 모니터링
    # 학습 과정에서 손실(logloss)이나 성능(AUC)을 확인 가능
    # 그래프/로그로 학습 추세 확인 → 과적합 여부 판단
    
    eval_metric="logloss", # 손실 함수
    random_state=42
)
model.fit(X_train, y_train)

# 5. 평가
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]

print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))

#label 0은 뇌출혈이 일어나지 않음(비출혈), 1은 출혈

#precision 전체에서 비출혈 예측은 50%
#recall 실제 비출혈 중 비출혈이라고 맞춘 건 6% -> 모델이 비출혈을 거의 맞추고 있지 못함
#f1-score 평균 성능 11%

#출혈이라고 예측한 확률은 68%
#출혈 중 실제 출혈이라고 맞춘 건 97%
#평균 성능 80%

#모델이 뇌출혈 예측에만 정확도가 높고 비출혈은 정확하게 예측하지 못함


              precision    recall  f1-score   support

           0       0.50      0.06      0.11        33
           1       0.68      0.97      0.80        67

    accuracy                           0.67       100
   macro avg       0.59      0.52      0.45       100
weighted avg       0.62      0.67      0.57       100

ROC-AUC: 0.6956128448665763


# 하이퍼 파라미터 튜닝

In [9]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

# ---------------------------
# 1. XGBoost 기본 모델 정의
# ---------------------------
xgb = XGBClassifier(
    eval_metric="logloss",     # 학습 모니터링용 손실 함수
    random_state=42
)

# ---------------------------
# 2. 탐색할 하이퍼파라미터 범위
# ---------------------------
param_grid = {
    'n_estimators': [100, 200, 300],    # 트리 개수
    'max_depth': [3, 4, 5],             # 트리 최대 깊이
    'learning_rate': [0.01, 0.05, 0.1], # 학습률
    'subsample': [0.7, 0.8, 1.0],       # 샘플링 비율
    'colsample_bytree': [0.7, 0.8, 1.0] # 특성 샘플링 비율
}

# ---------------------------
# 3. GridSearchCV 설정
# ---------------------------
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='roc_auc',  # 성능 기준: ROC-AUC
    cv=3,               # 3-겹 교차검증
    verbose=1,          # 진행 상황 출력
    n_jobs=-1           # CPU 코어 모두 사용
)

# ---------------------------
# 4. 그리드 탐색 실행
# ---------------------------
grid_search.fit(X_train, y_train)
# 243 조합 × 3 folds → 729번 학습 진행

# ---------------------------
# 5. 최적 파라미터 및 성능 확인
# ---------------------------
print("최적 하이퍼파라미터:", grid_search.best_params_)
print("최적 성능 (ROC-AUC):", grid_search.best_score_)


Fitting 3 folds for each of 243 candidates, totalling 729 fits
최적 하이퍼파라미터: {'colsample_bytree': 0.7, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
최적 성능 (ROC-AUC): 0.6663990088147392


# 클래스 불균형 성능 향상 방법이 필요 - 직접